# A First Look At PyROOT

This is an example of what you might do when you first look at some root files that you've never seen before. In this exercise, we will imagine that we're just getting started on some sort of dijet resonance analysis. Luckily, your signal, backgrounds, and data have all been ntuplized already, and the output ntuples are located on Fermilab eos at: 

`/store/user/hats/PyRoot/2017/qcd_samples`

In Jupyter, we can execute shell commands by prepending the usual commands with `!`

The following command is actually two steps:

* The `xrdfs` command lists (AKA `ls`) the files stored on the USA-based xrootd redirector `cmsxrootd.fnal.gov` in the directory `/store/user/hats/PyRoot/2017/qcd_samples`
* That command is "piped" (via the `|` symbol) into `sort -u`, which removes duplicate lines

We have to remove duplicate lines because `xrdfs` has an annoying habit of listing the file N number of times where N is the number of physical backend servers.

In [ ]:
!xrdfs root://xrootd.accre.vanderbilt.edu/ ls /store/user/hats/PyRoot/2017/qcd_samples | sort -u

Let's see our signal samples, which are in `shuffled_sig`:

In [ ]:
!xrdfs root://xrootd.accre.vanderbilt.edu/ ls /store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig | sort | uniq

As you can tell, these commands take some time to return, 

Let's use pyROOT to take a look at the basic structure of the file and the branches therein.

> Note:
>   One could do all of this in an interactive python session, using 
>
>   `python -i`
>
>  Interactive pyROOT is very useful for quickly looking at ROOT files and other tasks, such as formatting ROOT graphics. If you are interested in this, please ask the instructor for a demonstration of where this can come in handy (e.g. reformatting plots in interactive pyROOT while a TCanvas is open).

Commonly, pyROOT scripts start out with

`from ROOT import *`

but this is not allowed in JupyROOT. However, we can still save some typing by referring to ROOT with a shorter name:

In [ ]:
from ROOT import *

In [ ]:
import ROOT as r

Once we've loaded ROOT, we can load our files remotely, without having to transfer them to our local machine first.

If you get an `Auth Failed` error, make sure that you have a valid grid proxy with `voms-proxy-init -voms cms -valid 192:0`

(note, we're hardcoding a specific host with this file here -- this is due to a bug in the current xrootd server version, which should be fixed soon)

In [ ]:
# Note the double-slash ("//") between the hostname and store. This is required.
firstLookFile = r.TFile.Open("root://xrootd.accre.vanderbilt.edu//store/user/hats/PyRoot/2017/qcd_samples/shuffled_sig/sig_shuffled_0.root")


In [ ]:
firstLookFile.ls()

In [ ]:
firstLookTree = firstLookFile.Get("tree")

Already, we're benefitting from pyROOT's syntax. Note that a similar step in an interactive ROOT session would require a bit more effort:

`TTree* firstLookTree = (TTree*) firstLookFile->Get("tree")`

Let's use a quick python loop to list all the branches in our tree.

In [ ]:
for branch in firstLookTree.GetListOfBranches():
    print branch.GetName()

Our analysis is going to look for dijet resonances using AK4 jets. So to get a feel for the data, let's look at what the AK4 jet pT spectrum looks like.

In [ ]:
can = r.TCanvas()      
firstLookTree.Draw("jetAK4_pt")
can.Draw()

> Note that in interactive pyROOT, one can simply call `firstLookTree.Draw("jetAK4_pt")` without bothering to explicitly instantiate a TCanvas and call TCanvas.Draw() on it

Let's do some quick changes to the plot. `r.gStyle` gives us access to common style changes for ROOT graphics.

In [ ]:
r.gStyle.SetOptStat(0)
can.SetLogy()
can.Draw()

Let's quickly demonstrate python's introspection features. ROOT automatically builds a lot of things into a TCanvas that might not be obvious when you simply call `TTree.Draw('branchName')`, so let's see what's in our canvas and what methods we could call on it. And let's make the output more readable while we're at it. 

In [ ]:
import inspect
from pprint import pprint
for primitive in can.GetListOfPrimitives():
    print primitive, pprint(inspect.getmembers(primitive, predicate=inspect.ismethod))

Now that we know what's in our trees and gotten a quick look to sanity-check the data, we'll get started trying to tackle the many gigs of data that are at hand. pyROOT is not the right tool for crunching this much data, so let's move on to preparing a C++ class to do the number crunching.

In [ ]:
firstLookTree.MakeClass("hatsTrees")

Let's see what this made for us:

In [ ]:
!ls -lrth

Our aim is to use this C++ class to do the CPU-intensive work of our analysis, but to keep it as a "plugin" inside our pyroot framework.

>Please move on to the notebook `prepareCppClass.ipynb`